In [ ]:
!rm -rf Monocular-Metric-Depth-Estimation
!git clone https://github.com/Thorfast191/Monocular-Metric-Depth-Estimation.git
%cd Monocular-Metric-Depth-Estimation

!pip install -r requirements.txt
!pip install torch torchvision

import sys
sys.path.append('/content/Monocular-Metric-Depth-Estimation')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!bash scripts/download_kitti.sh

In [ ]:
import yaml
import torch
from torch.utils.data import DataLoader

from datasets.kitti_dataset import KITTIDataset
from models.depth_model import DepthModel
from trainers.trainer import Trainer
from utils.logger import Logger  # if exists

In [ ]:
with open("configs/kitti.yaml", "r") as f:
    cfg = yaml.safe_load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dataset = KITTIDataset(
    root=cfg["data_root"],
    split="train"
)

train_loader = DataLoader(
    train_dataset,
    batch_size=cfg["train"]["batch_size"],
    shuffle=True,
    num_workers=cfg["train"]["num_workers"]
)

In [ ]:
model = DepthModel().to(device)

In [ ]:
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=cfg["train"]["lr"]
)

In [ ]:
class DummyLogger:
    def log(self, data):
        print(data)

logger = DummyLogger()

In [ ]:
trainer = Trainer(
    model=model,
    loader=train_loader,
    optimizer=optimizer,
    logger=logger,
    device=device
)

In [ ]:
trainer.train(cfg["train"]["epochs"])